<a href="https://colab.research.google.com/github/jcmachicao/gdmk_uc__reportes_edusights/blob/main/ucont__mprecitivo_train_202503_tensorready.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrenamiento de Red Neuronal Clasificación Binaria PyTorch

## Preparación

In [ ]:
!pip install torch torchvision

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from torchvision import datasets, transforms
import os

In [ ]:
if torch.cuda.is_available():
    print("GPU is available.")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available.")

GPU is available.
Device name: Tesla T4


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
!nvidia-smi

Thu Mar  6 11:30:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
data_path = 'drive/My Drive/00 2025_all/2025 Proyectos/2025__Edusights_JCMV/archivos/aplicativo 2024 edusights/edusights_app/'
os.listdir(data_path + 'archivos3')

['202510_base_al0503.xlsx',
 '202510_base_al0503.pkl',
 'target_df.pkl',
 'X_train_tensor.pt',
 'X_train_tensor_sample.pt',
 'y_train_tensor.pt',
 'resultados_df_new.xlsx']

## Rerun or Training

In [20]:
X_tot = torch.load(data_path + 'archivos3/' + 'X_train_tensor.pt')
y_tot = torch.load(data_path + 'archivos3/' + 'y_train_tensor.pt')
X_tot.shape, y_tot.shape

<ipython-input-20-310b68938b7d>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X_tot = torch.load(data_path + 'archivos3/' + 'X_train_tensor.pt')
<ipython-input-20-310b689

(torch.Size([8020, 149]), torch.Size([8020]))

In [21]:
FRAC_TEST = 0.10

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_tot, y_tot, test_size=FRAC_TEST, random_state=None)

In [23]:
X_train.shape

torch.Size([7218, 149])

In [24]:
X_train.max(), X_train.min()

(tensor(74.), tensor(0.))

In [ ]:
X_train

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000, 13.4400, 31.0000],
        [ 0.0000,  0.0000,  1.0000,  ...,  0.0000, 17.4500, 25.0000],
        [ 0.0000,  0.0000,  1.0000,  ...,  0.0000, 15.0500, 38.0000],
        ...,
        [ 0.0000,  1.0000,  0.0000,  ...,  0.0000, 10.8000, 23.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, 14.4700, 24.0000],
        [ 0.0000,  0.0000,  1.0000,  ...,  0.0000, 12.0000, 30.0000]])

In [ ]:
(y_train > 0).sum()

tensor(3864)

In [ ]:
torch.save(X_train, data_path + 'archivos/X_train.pt')
torch.save(y_train, data_path + 'archivos/y_train.pt')
torch.save(X_test, data_path + 'archivos/X_test.pt')
torch.save(y_test, data_path + 'archivos/y_test.pt')

In [ ]:
X_train_tensor = X_train.clone().requires_grad_(True).to(device)  # Create a tensor with requires_grad=True
X_test_tensor = X_test.clone().requires_grad_(True).to(device)  # Create a tensor with requires_grad=True
y_train_tensor = y_train.clone().requires_grad_(True).to(device)  # Create a tensor with requires_grad=True
y_test_tensor = y_test.clone().requires_grad_(True).to(device)  # Create a tensor with requires_grad=True

In [ ]:
# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(X_train_tensor.shape, y_train_tensor.shape)
print(X_test_tensor.shape, y_test_tensor.shape)
print(len(train_loader))

torch.Size([6817, 149]) torch.Size([6817])
torch.Size([1203, 149]) torch.Size([1203])
214


In [ ]:
class ImprovedNN1(nn.Module):
    def __init__(self, input_size):
        super(ImprovedNN1, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 32)
        self.fc5 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.2)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)
        self.relu = nn.ReLU()
        self.leaky_relu = nn.LeakyReLU(0.01)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.leaky_relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.relu(self.fc4(x))
        x = self.sigmoid(self.fc5(x))
        return x

In [ ]:
class ClassificationNN3(nn.Module):
    def __init__(self, input_size):
        super(ClassificationNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 2)  # Output layer (2 classes)
        self.dropout = nn.Dropout(0.05)  # Reduced dropout
        self.leaky_relu = nn.LeakyReLU(0.01)  # Consistent activation

    def forward(self, x):
        x = self.leaky_relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.bn3(self.fc3(x)))
        x = self.leaky_relu(self.fc4(x))
        x = self.fc5(x)  # No activation here (logits for CrossEntropyLoss)
        return x

# Initialize model
input_size = 149  # Ensure this matches your dataset
model = ClassificationNN3(input_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.9)

NameError: name 'nn' is not defined

In [ ]:
class ClassificationNN(nn.Module):
    def __init__(self, input_size):
        super(ClassificationNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 32)
        self.fc5 = nn.Linear(32, 2)  # Output layer with 2 neurons for 2 classes
        self.dropout = nn.Dropout(0.2)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)
        self.relu = nn.ReLU()
        self.leaky_relu = nn.LeakyReLU(0.01)
        # Removed sigmoid as we'll use CrossEntropyLoss
        # which combines LogSoftmax and NLLLoss

    def forward(self, x):
        x = self.leaky_relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.relu(self.fc4(x))
        x = self.fc5(x)  # No sigmoid here
        return x

### Tensores

In [ ]:
input_size = X_train.shape[1]
num_classes = len(np.unique(y_tot))
print('Tamaño de Entrada: ', input_size, 'Número de Clases de Salida: ', num_classes)
#model = ImprovedNN1(input_size).to(device)
model = ClassificationNN(input_size=input_size).to(device)

Tamaño de Entrada:  149 Número de Clases de Salida:  2


In [ ]:
next(model.parameters()).device

device(type='cuda', index=0)

In [ ]:
criterion = nn.BCELoss()  # or nn.BCEWithLogitsLoss() if you remove the sigmoid from the model
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

In [ ]:
# For multi-class classification
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
num_classes = len(np.unique(y_tot)) # Get the number of unique classes from y_tot

In [ ]:
NUM_EPOCHS = 3000

In [ ]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=NUM_EPOCHS):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            '''
            outputs = outputs.squeeze()  # Squeeze to remove extra dimensions
            loss = criterion(outputs, labels.float())  # Ensure labels are floats
            '''
            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()
            scheduler.step()
            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

In [ ]:
def evaluate_model_v2(model, test_data, test_labels):
    model.eval()
    with torch.no_grad():
        outputs = model(test_data)

    '''
    outputs_show = outputs.cpu().numpy().flatten()
    outputs_show[outputs_show > 0.51] = 1.0
    outputs_show[outputs_show < 0.49] = 0.0
    '''
    _, predicted = torch.max(outputs.data, 1)
    outputs_show = predicted.cpu().numpy()
    test_labels_np = test_labels.detach().cpu().numpy()

    accuracy = 100 * (np.sum(outputs_show == test_labels_np) / len(test_labels_np))
    print(f'Precisión del Modelo en Data de Prueba: {accuracy:.2f}%')

    results_df = pd.DataFrame({
        'Real': test_labels_np,
        'Pred': outputs_show
    })

    return results_df, accuracy

### Training

In [ ]:
# Train and evaluate the model
train_model(model, train_loader, criterion, optimizer, num_epochs=NUM_EPOCHS)

Epoch [971/1000], Loss: 0.6998
Epoch [972/1000], Loss: 0.7003
Epoch [973/1000], Loss: 0.6988
Epoch [974/1000], Loss: 0.6987
Epoch [975/1000], Loss: 0.7005
Epoch [976/1000], Loss: 0.7011
Epoch [977/1000], Loss: 0.7013
Epoch [978/1000], Loss: 0.6999
Epoch [979/1000], Loss: 0.7012
Epoch [980/1000], Loss: 0.7010
Epoch [981/1000], Loss: 0.7011
Epoch [982/1000], Loss: 0.6994
Epoch [983/1000], Loss: 0.7006
Epoch [984/1000], Loss: 0.7000
Epoch [985/1000], Loss: 0.7004
Epoch [986/1000], Loss: 0.6998
Epoch [987/1000], Loss: 0.7006
Epoch [988/1000], Loss: 0.7018
Epoch [989/1000], Loss: 0.6990
Epoch [990/1000], Loss: 0.7011
Epoch [991/1000], Loss: 0.7008
Epoch [992/1000], Loss: 0.7001
Epoch [993/1000], Loss: 0.6990
Epoch [994/1000], Loss: 0.7000
Epoch [995/1000], Loss: 0.6987
Epoch [996/1000], Loss: 0.6985
Epoch [997/1000], Loss: 0.6987
Epoch [998/1000], Loss: 0.7015
Epoch [999/1000], Loss: 0.6985
Epoch [1000/1000], Loss: 0.7011


In [ ]:
X_test_tensor.shape, y_test_tensor.shape

(torch.Size([1604, 149]), torch.Size([1604]))

In [ ]:
results_df, accuracy = evaluate_model_v2(model, X_test_tensor, y_test_tensor)

Precisión del Modelo en Data de Prueba: 47.57%


In [ ]:
VERSION_NAME = 'S3_CON_100'

In [ ]:
results_df.to_excel(data_path + f'results_{VERSION_NAME}.xlsx')

In [ ]:
accuracy

47.56857855361596

In [ ]:
model_name = f'edusights_20240702_model_{VERSION_NAME}.pth'
dict_name = f'edusights_20240702_state_dict_{VERSION_NAME}.pth'
torch.save(model, data_path + 'modelos/' + model_name)
torch.save(model.state_dict(), data_path + 'modelos/' + dict_name)